In [1]:
#model: https://github.com/kristpapadopoulos/seriesnet/blob/master/seriesnet.py
#causalconv1: https://github.com/pytorch/pytorch/issues/1333

import torch
import numpy as np
import torch.nn as nn
from math import sqrt
import torch.nn.functional as F
import pytorch_lightning as pl

def weight_init(m):
    if isinstance(m, nn.Conv1d):
        torch.nn.init.trunc_normal_(m.weight, 0.0, 0.05)

class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,nb_filter,filter_length,dilation,device):
        self.__padding = ((filter_length - 1) * dilation)
        
        #(batch, in_channels, in_length) 
        super(CausalConv1d, self).__init__(7,out_channels = nb_filter,kernel_size=filter_length,
            stride=1,padding=self.__padding,dilation=dilation,groups=1,bias=False,device=device)

    def forward(self, x_input,cond): 
        """ x_input = [32,7,6], cond=[32,2]"""
        conv1d_out = super(CausalConv1d, self).forward(x_input)
        conv1d_out = conv1d_out[:, :, :-self.__padding] if self.__padding != 0 else conv1d_out
        return conv1d_out

class DC_CNN_Block(torch.nn.Module):
    def __init__(self,nb_filter, filter_length, dilation, num_features):
        super(DC_CNN_Block, self).__init__()
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.nb_filter = nb_filter
        self.filter_length= filter_length
        self.dilation = dilation
        self.num_features = num_features

        self.causal = CausalConv1d(self.nb_filter,self.filter_length,self.dilation,device=device)
        self.skip_out = nn.Conv1d(6,out_channels=self.num_features,kernel_size=1,stride=6,bias=False,device=device)
        self.network_in = nn.Conv1d(6,out_channels=self.num_features,kernel_size=1,stride=6,bias=False,device=device)
        
        # Initialize weights
        self.causal.apply(weight_init)
        self.skip_out.apply(weight_init)
        self.network_in.apply(weight_init)

    def forward(self, x_input,cond):
        dev = x_input.device

        residual = x_input #[32,6,7]
        x = x_input.permute(0,2,1).to(dev) #[32,7,6]

        #[32,7,6] => [32,32,6]
        layer_out = self.causal(x,cond).to(dev)
        layer_out = layer_out.permute(0,2,1).to(dev) #[32, 6, 32]
        layer_out = F.selu(layer_out) #[32, 6, 32]

        #[32, 6, 32] => [32, 7, 6]
        skip_out = self.skip_out(layer_out)
        skip_out = skip_out.permute(0,2,1).to(dev) #[32, 6, 7]

        #[32, 6, 32] => [32, 7, 6]
        network_in = self.network_in(layer_out)
        network_in = network_in.permute(0,2,1).to(dev) #[32, 6, 7]

        network_out = residual + network_in
        return network_out, skip_out



class CausalModel(pl.LightningModule):
    def __init__(self, w_decay,dropout,alpha,gamma,input_seq_length,output_seq_length,
                 num_features,lr,num_conditions, path,feature_list,
                 net,in_channels,out_channels,kernel_size,stride,dilation,groups,bias):

        super(CausalModel,self).__init__()
        self.w_decay = w_decay
        self.dropout = dropout
        self.alpha = alpha
        self.gamma = gamma
        self.input_seq_length = input_seq_length
        self.output_seq_length = output_seq_length
        self.num_features = num_features
        self.lr = lr
        self.num_conditions = num_conditions
        self.path = path
        self.feature_list = feature_list
        self.net = net
        self.save_hyperparameters()

        self.block_1 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=1, num_features=self.num_features)
        self.block_2 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=2, num_features=self.num_features)
        self.block_3 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=4, num_features=self.num_features)
        self.block_4 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=8, num_features=self.num_features)
        self.block_5 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=16, num_features=self.num_features)
        self.block_6 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=32, num_features=self.num_features)
        self.block_7 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=64, num_features=self.num_features)
        self.l21 = nn.Conv1d(self.num_features,out_channels=self.num_features,kernel_size=1,stride=1,bias=False)

        # Initialize weights 
        self.block_1.apply(weight_init)
        self.block_2.apply(weight_init)
        self.block_3.apply(weight_init)
        self.block_4.apply(weight_init)
        self.block_5.apply(weight_init)
        self.block_6.apply(weight_init)
        self.block_7.apply(weight_init)
        self.l21.apply(weight_init)

    def forward(self,x_input,cond): 
        """
        input sequence: [batch_size, input_seq_length,num_features]
        condition: [batch_size,num_conditions]
        """
        dev = x_input.device #[32,6,7]

        l1a, l1b = self.block_1(x_input,cond)
        l2a, l2b = self.block_2(l1a,cond)
        l3a, l3b = self.block_3(l2a,cond)
        l4a, l4b = self.block_4(l3a,cond)
        l5a, l5b = self.block_5(l4a,cond)
        l6a, l6b = self.block_6(l5a,cond)
        l6b = nn.Dropout(0.8)(l6b)
        l7a, l7b = self.block_7(l6a,cond)
        l7b = nn.Dropout(0.8)(l7b)
        l8 = l1b + l2b + l3b + l4b + l5b + l6b + l7b
        l9 = F.relu(l8)

        l9 = l9.permute(0,2,1).to(dev)
        l21 = self.l21(l9)
        l21 = l21.permute(0,2,1).to(dev)

        return l21

    def training_step(self, batch, batch_idx):
        target_in, target_out,condition, mask = batch
        condition = condition[:,:self.num_conditions] #settings to number of conditions
        target_in = torch.tensor(target_in, dtype=torch.float32).to(target_in.device)#[32, 6, 7]
        target_out = torch.tensor(target_out, dtype=torch.float32).to(target_out.device)#[32, 6, 7]        
        
        y_pred = self(target_in,condition)#[32, 6, 7]
        synth_mask = y_pred
        real_mask = target_out
    
        rmse = rmse_loss(synth_mask,real_mask)
        self.log("loss_train", rmse,on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {"loss":rmse,"past":target_in,"ytrue":target_out,"ypred":y_pred,"conditions":condition, "mask":mask}

    def validation_step(self, batch, batch_idx):
        target_in, target_out,condition, mask = batch
        condition = condition[:,:self.num_conditions] #settings to number of conditions
        target_in = torch.tensor(target_in, dtype=torch.float32).to(target_in.device)
        target_out = torch.tensor(target_out, dtype=torch.float32).to(target_out.device)
        
        y_pred = self(target_in,condition)    
        synth_mask = y_pred
        real_mask = target_out

        rmse = rmse_loss(synth_mask,real_mask)
        self.log("loss_val",rmse)
        return {"loss":rmse,"past":target_in,"ytrue":target_out,"ypred":y_pred,"conditions":condition, "mask":mask}

    def configure_optimizers(self):
        #weight_decay sustituye a: kernel_regularizer=l2(l2_layer_reg))
        optimizer = torch.optim.Adam(self.parameters(),lr=0.00075,betas=(0.9, 0.999),weight_decay=0.0)        
        return {"optimizer": optimizer,"monitor": "loss",}

/home/blanca/anaconda3/envs/py38/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
